In [ ]:
# mount it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://gist.githubusercontent.com/Yegorov/dc61c42aa4e89e139cd8248f59af6b3e/raw/20ac954e202fe6a038c2b4bb476703c02fe0df87/ya.py
!chmod +x ya.py
!./ya.py https://disk.yandex.ru/d/f3s_caF7M7kA3A ./

In [ ]:
#!cp /content/drive/MyDrive/colab_export/omsk_forest/import/test_dataset_test.zip test.zip
!unzip test_dataset_test.zip

In [ ]:
!ls

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install

In [ ]:
%cd ..
!ls

In [ ]:
!cp /content/drive/MyDrive/colab_export/omsk_forest/export/model_4/exp/weights/best.pt best.pt

In [ ]:
#!mkdir crop result

In [ ]:
!ls

best.pt  crop  drive  result  sample_data  test  test.zip  yolov5


In [ ]:
from pathlib import Path
from typing import List
from PIL import Image

In [ ]:
sample_image = Path('test')
sample = Path('test')
labels = Path('result/exp/labels')

In [ ]:
def get_files(files_dir: Path, suff: str = 'png') -> List[Path]:
    return sorted(Path(files_dir).glob(f'*.{suff}'))

In [ ]:
sample_list = get_files(sample_image, 'JPG')

In [ ]:
def crop_image(infile: Path, chopsize: int, crop_dir: Path):
   img = Image.open(infile)
   width, height = img.size

   # Save Chops of original image
   for x0 in range(0, width, chopsize):
      for y0 in range(0, height, chopsize):
         box = (x0, y0,
               x0+chopsize if x0+chopsize <  width else  width - 1,
               y0+chopsize if y0+chopsize < height else height - 1)
         #print('%s %s' % (infile, box))
         file_name = crop_dir / infile.stem
         img.crop(box).save('%s.x.%03d.y.%03d.jpg' % (file_name, x0, y0))

In [ ]:
#for image in sample_list:
    #size = 256
    #crop_image(image, size, sample)

In [ ]:
!python yolov5/detect.py --weights best.pt --source test/ --save-txt --project result/

detect: weights=['best.pt'], source=test/, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=result/, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-210-gf9bb984 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 276 layers, 35248920 parameters, 0 gradients, 48.9 GFLOPs
image 1/3435 /content/test/1.JPG: 384x640 (no detections), 25.6ms
image 2/3435 /content/test/10.JPG: 384x640 (no detections), 25.4ms
image 3/3435 /content/test/100.JPG: 384x640 (no detections), 25.3ms
image 4/3435 /content/test/1000.JPG: 384x640 (no detections), 25.3ms
image 5/3435 /content/test/1001.JPG: 384x640 (no detections), 25.3ms
image 6/3435 /content/test/1002.JPG: 384x

In [ ]:
!cp -r result/exp/ /content/drive/MyDrive/colab_export/omsk_forest/export/detect/30_10_22_19_26/

In [ ]:
!ls result/exp/labels

1000.txt  1405.txt  1778.txt  2162.txt	2550.txt  2991.txt  3355.txt  669.txt
1002.txt  1406.txt  1781.txt  2169.txt	2552.txt  2993.txt  3358.txt  66.txt
1005.txt  1415.txt  1792.txt  216.txt	2561.txt  2997.txt  3359.txt  677.txt
1008.txt  1416.txt  1793.txt  2172.txt	2565.txt  2998.txt  3360.txt  67.txt
100.txt   1418.txt  1794.txt  2177.txt	2567.txt  29.txt    3361.txt  681.txt
1012.txt  141.txt   1795.txt  217.txt	2570.txt  3000.txt  3363.txt  683.txt
1013.txt  1423.txt  1796.txt  2182.txt	2572.txt  3001.txt  3366.txt  686.txt
1014.txt  1424.txt  1797.txt  2185.txt	2573.txt  3002.txt  3367.txt  687.txt
1018.txt  1428.txt  17.txt    218.txt	2576.txt  3005.txt  3368.txt  68.txt
1020.txt  1429.txt  1801.txt  2192.txt	2579.txt  3010.txt  3370.txt  691.txt
1022.txt  142.txt   1802.txt  2193.txt	2581.txt  3011.txt  3374.txt  693.txt
1023.txt  1432.txt  1803.txt  2197.txt	2583.txt  3016.txt  3377.txt  697.txt
1024.txt  1437.txt  1804.txt  21.txt	2585.txt  3018.txt  3378.txt  699.txt
1026.tx

In [ ]:
def get_files(files_dir: Path, suff: str = 'png') -> List[Path]:
    return sorted(Path(files_dir).glob(f'*.{suff}'))

In [ ]:
labels_list = get_files(labels, 'txt')

In [ ]:
labels_list

In [ ]:
def str_to_float(labels_list: List[List[str]]) -> List[List[float]]:
    float_labels = []
    for label in labels_list:
        int_label = [float(x) for x in label]
        float_labels.append(int_label)
    return float_labels

In [ ]:
def read_txt(label_file: Path) -> List[List[float]]:
    # функция возвращает список боксов с координатами в формате
    # [номер класса объекта] [x - центр] [y -центр] [ширина от центра] [высота от центра]
    # значения координат в диапазоне от 0 до 1 (доля от общих рамзеров картинки)
    with open(label_file, 'r') as labels:
        labels_list = labels.readlines()
        labels_list = [row.replace('\n', '') for row in labels_list]
        labels_list = [row.split(' ') for row in labels_list]
        return str_to_float(labels_list)

In [ ]:
def fraction_to_pixel(labels: List[List[float]], x: int = 0, y: int = 0, size=(256,256)) -> List[List[int]]:
    width, height = size
    #width = width + x
    #height = height + y
    new_labels = []
    for label in labels:
        _, x_center, y_center, obj_width, obj_height = label
        x_center = int(x_center * width + x)
        obj_width = obj_width * width
        y_center = int(y_center * height + y)
        obj_height = obj_height * height
        radius = int((obj_width + obj_height) / 2)
        new_labels.append({'cx': x_center, 'cy': y_center, 'r': radius})
    return new_labels

In [ ]:
def get_x_y(label_file: Path):
    # .x.%03d.y.%03d.jpg
    name = label_file.stem    
    name_list = name.split('.')
    if len(name_list) < 2:
      return name, 0, 0
    parent_name = name_list[0]
    x_min = int(name_list[2])
    y_min = int(name_list[4])
    return parent_name, x_min, y_min

In [ ]:
't'.split('.')


['t']

In [ ]:
def get_size(label_name, suff='.JPG'):
    try:
      image_file = sample / f'{label_name}{suff}'
      image = Image.open(image_file)
    except:
      suff = '.jpg'
      image_file = sample / f'{label_name}{suff}'
      image = Image.open(image_file)    
    return image.size

In [ ]:
detect_labels = []
for label_file in labels_list:
    labels = read_txt(label_file)
    parent_name, x_min, y_min = get_x_y(label_file)
    size = get_size(label_file.stem)
    pixel_labels = fraction_to_pixel(labels, x=x_min, y=y_min, size=size)
    real_labels = [parent_name]
    real_labels.extend(pixel_labels)
    detect_labels.append(real_labels)
    print(real_labels)
    #break

['103', {'cx': 275, 'cy': 98, 'r': 20}]
['1039', {'cx': 22, 'cy': 9, 'r': 33}, {'cx': 674, 'cy': 321, 'r': 26}]
['1054', {'cx': 3596, 'cy': 1624, 'r': 110}, {'cx': 3732, 'cy': 1933, 'r': 104}]
['1068', {'cx': 386, 'cy': 563, 'r': 30}]
['1146', {'cx': 122, 'cy': 132, 'r': 23}]
['1168', {'cx': 259, 'cy': 385, 'r': 31}, {'cx': 698, 'cy': 72, 'r': 23}]
['122', {'cx': 938, 'cy': 237, 'r': 57}]
['1231', {'cx': 2574, 'cy': 1952, 'r': 102}]
['1236', {'cx': 122, 'cy': 132, 'r': 23}]
['1240', {'cx': 531, 'cy': 18, 'r': 25}]
['1279', {'cx': 605, 'cy': 2775, 'r': 93}, {'cx': 2928, 'cy': 692, 'r': 102}]
['1297', {'cx': 467, 'cy': 504, 'r': 28}]
['1306', {'cx': 424, 'cy': 341, 'r': 38}]
['1326', {'cx': 206, 'cy': 336, 'r': 31}]
['1329', {'cx': 424, 'cy': 341, 'r': 38}]
['1350', {'cx': 458, 'cy': 113, 'r': 25}]
['1363', {'cx': 647, 'cy': 6, 'r': 18}]
['1370', {'cx': 309, 'cy': 276, 'r': 23}]
['1381', {'cx': 22, 'cy': 9, 'r': 33}, {'cx': 674, 'cy': 322, 'r': 26}]
['139', {'cx': 2431, 'cy': 950, 'r': 1

In [ ]:
!mkdir csv

In [ ]:
import pandas as pd

labels_df = pd.DataFrame(columns=['labels'])
labels_df['labels'] = detect_labels
labels_df.to_csv('csv/labels1.csv', sep=',')

In [ ]:
!ls csv

labels.csv


In [ ]:
!cp -r csv /content/drive/MyDrive/colab_export/omsk_forest/export/csv_26_10_22/